In [1]:
!pip install plotly

You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.7/bin/python3.7 -m pip install --upgrade pip' command.


In [1]:
df = pd.read_csv('data_analysis/csv_files/all_seeds.csv')

df
df['summed_test_acc'] = df[['ANLI_test_acc_epoch', 'SNLI_test_acc_epoch', 'WANLI_test_acc_epoch']].mean(axis=1)
df = df.groupby('train_sets').agg([np.std, np.mean])
df['summed_test_acc']
# df

NameError: name 'pd' is not defined

In [749]:
import plotly
import pandas as pd
import plotly.express as px
import pandas as pd
from scipy.special import softmax
import numpy as np

path = 'data_analysis/csv_files/all_seeds.csv'

def generate_plot(filepath, test_column, plot_title, stds=None):

    df = pd.read_csv(filepath)
    df['summed_test_acc'] = df[['ANLI_test_acc_epoch', 'SNLI_test_acc_epoch', 'WANLI_test_acc_epoch']].mean(axis=1)
    df = df.groupby('train_sets').agg([np.std, np.mean])

    SNLI_fracs = []
    ANLI_fracs = []
    WANLI_fracs = []

    test_array = []
    stds = []

    for i in range(len(df)):
        row = df.iloc[i]

        name = row.name.replace('"', '').replace("]", '').replace("[", '').replace(',',' ').split(' ')
        if float(name[0].split('_')[1]) == 0.33333:
            continue
        
        else:
            SNLI_fracs.append(float(name[0].split('_')[1]))
            ANLI_fracs.append(float(name[1].split('_')[1]))
            WANLI_fracs.append(float(name[2].split('_')[1]))

            test_array.append(row[test_column]['mean'])
            stds.append(row[test_column]['std'])

    test_array =  [round(score, 5) for score in test_array]
    test_sizes = [round((score**3), 3) for score in stds]
    
    test_sizes = [4 for val in test_sizes]
    
    print(' '.join(plot_title.split(' ')[1:]))
    
    df = pd.DataFrame({'SNLI':SNLI_fracs, 
                       'ANLI':ANLI_fracs, 
                       'WANLI':WANLI_fracs, 
                       '{}'.format(' '.join(plot_title.split(' ')[1:])):test_array, 
                       "test_sizes": test_sizes})
    
    fig = px.scatter_ternary(df, 
                             a="SNLI", 
                             b="ANLI", 
                             c="WANLI", 
                             size="test_sizes", 
                             color='{}'.format(' '.join(plot_title.split(' ')[1:])),
                             range_color=[min(test_array),max(test_array)],
                             opacity=1)

    atickvals = ['0% ','25% ', '50% ', '75% ', '100% ']
    btickvals = ['0%','25%', '50%', '75%', '100%']
    ctickvals = [' 0%',' 25%', ' 50%', ' 75%', ' 100%']
#     tickvals = [0.0, 0.25, 0.5, 0.75, 1.0]
    ticksize = 10
    titlesize = 20

    fig.update_ternaries(aaxis_tickvals=[0, 0.25, 0.5, 0.75, 1.0])
    fig.update_ternaries(aaxis_ticktext=atickvals)
    fig.update_ternaries(aaxis_tickfont={'size':ticksize})
    fig.update_ternaries(aaxis_title={'font':{'size':titlesize}})
    fig.update_ternaries(aaxis_title={'text':''})
    fig.update_ternaries(aaxis_tickangle=0)

    fig.update_ternaries(baxis_tickvals=[0, 0.25, 0.5, 0.75, 1.0])
    fig.update_ternaries(baxis_ticktext=btickvals)
    fig.update_ternaries(baxis_tickfont={'size':ticksize})
    fig.update_ternaries(baxis_title={'font':{'size':titlesize}})
    fig.update_ternaries(baxis_title={'text':''})
    fig.update_ternaries(baxis_tickangle=0)

    fig.update_ternaries(caxis_tickvals=[0, 0.25, 0.5, 0.75, 1.0])
    fig.update_ternaries(caxis_ticktext=ctickvals)
    fig.update_ternaries(caxis_tickfont={'size':ticksize})
    fig.update_ternaries(caxis_title={'font':{'size':titlesize}})
    fig.update_ternaries(caxis_title={'text':''})
    fig.update_ternaries(caxis_tickangle=0)

    fig.update_layout(
            autosize=True,
            height=350,
            width=350,
            margin=dict(
                l=5,
                r=5,
                b=5,
                t=20,
                pad=0
            ),
)
    
    fig.add_annotation(text="SNLI",
                  xref="paper", yref="paper",
                  x=0.1, y=0.6, showarrow=False,
                      font=dict(
            family="Open Sans",
            size=18
            ))
    
    fig.add_annotation(text="ANLI",
                  xref="paper", yref="paper",
                  x=0.5, y=-0.20, showarrow=False,
                      font=dict(
            family="Open Sans",
            size=18
            ))
    
    DTICK = round((max(test_array) - min(test_array))/5, 2)
    
    fig.add_annotation(text="WANLI",
                  xref="paper", yref="paper",
                  x=0.9, y=0.6, showarrow=False,
                      font=dict(
            family="Open Sans",
            size=18
            ))
    
    fig.update_layout(coloraxis=dict(colorbar=dict(orientation='h', 
                                                   thickness=20, 
                                                   x=0.48, 
                                                   y=-0.5, 
                                                   len=0.9, 
                                                   tick0 =round(min(test_array),2),  
                                                   dtick=DTICK,
                                                   ticks="outside")))#, tick0=0, dtick=0.1)))
    import plotly.graph_objects as go
    fig.add_trace(
    go.Scatterternary(a=[0.5, 0],
                      b=[0.5, 0.5],
                      mode='lines',
                      marker=dict(color='red'),
                      line=dict(width=2),
                      showlegend=False)
)
    fig.add_trace(
    go.Scatterternary(
                      b=[0, 0.5],
                      c=[0, 0],
                      mode='lines',
                      marker=dict(color='red'),
                      line=dict(width=6),
                      showlegend=False)
)
#     fig.add_trace(
#     go.Scatterternary(
#                       b=[0.75, 0.25],
#                       c=[0, 0.5,],
#                       mode='lines',
#                       marker=dict(color='red'),
#                       line=dict(width=2),
#                       showlegend=False)
# )

    #garbage graph
    fig2 = px.scatter(x=[0, 1, 2, 3, 4], y=[0, 1, 4, 9, 16])
    fig2.write_image('{}.pdf'.format(plot_title))
    plotly.io.write_image(fig, 'data_analysis/images/combinations/{}_2.pdf'.format(plot_title), format='pdf')

    fig.show()

In [750]:
SIZE = 50
# generate_plot(filepath=path, test_column='WANLI_test_acc_epoch', plot_title='WANLI Test Acc.')
# generate_plot(filepath=path, test_column='ANLI_test_acc_epoch', plot_title='ANLI Test Acc.')
# generate_plot(filepath=path, test_column='SNLI_test_acc_epoch', plot_title='SNLI Test Acc.')
generate_plot(filepath=path, test_column='MNLI_test_acc_epoch', plot_title='MNLI Test Acc.')
# generate_plot(filepath=path, test_column='summed_test_acc', plot_title='Agg. Test Acc.')

Test Acc.


In [183]:
import plotly.figure_factory as ff
import numpy as np
# Al = np.array([0. , 0. , 0., 0., 1./3, 1./3, 1./3, 2./3, 2./3, 1.])
# Cu = np.array([0., 1./3, 2./3, 1., 0., 1./3, 2./3, 0., 1./3, 0.])
# Y = 1 - Al - Cu
# # synthetic data for mixing enthalpy
# # See https://pycalphad.org/docs/latest/examples/TernaryExamples.html
# enthalpy = (Al - 0.01) * Cu * (Al - 0.52) * (Cu - 0.48) * (Y - 1)**2
fig = ff.create_ternary_contour(np.array([SNLI_fracs, ANLI_fracs, WANLI_fracs]), np.array(MNLI_test),
                                pole_labels=['SNLI', 'ANLI', 'WANLI'], showscale=True,
                                interp_mode='cartesian')

tickvals = ['','25%', '50%', '75%', '']
ticksize = 10
titlesize = 13

fig.update_ternaries(aaxis_tickvals=[0, 0.25, 0.5, 0.75, 1.0])
fig.update_ternaries(aaxis_ticktext=tickvals)
fig.update_ternaries(aaxis_tickfont={'size':ticksize})
fig.update_ternaries(aaxis_title={'font':{'size':titlesize}})
fig.update_ternaries(aaxis_tickangle=25)

fig.update_ternaries(baxis_tickvals=[0, 0.25, 0.5, 0.75, 1.0])
fig.update_ternaries(baxis_ticktext=tickvals)
fig.update_ternaries(baxis_tickfont={'size':ticksize})
fig.update_ternaries(baxis_title={'font':{'size':titlesize}})
fig.update_ternaries(baxis_tickangle=0)

fig.update_ternaries(caxis_tickvals=[0, 0.25, 0.5, 0.75, 1.0])
fig.update_ternaries(caxis_ticktext=tickvals)
fig.update_ternaries(caxis_tickfont={'size':ticksize})
fig.update_ternaries(caxis_title={'font':{'size':titlesize}})
fig.update_ternaries(caxis_tickangle=-25)

fig.show()

NameError: name 'SNLI_fracs' is not defined

In [ ]:
import skimage
print(skimage.__version__)